In [4]:
import psycopg2
import pandas as pd

# Connect to PostgreSQL database
def connect_to_postgres():
    conn = psycopg2.connect(
        host="localhost",       
        database="lombardia_air_quality", 
        user="airdata_user",    
        password="user"
    )
    return conn


In [5]:


def query_data(query, format='dataframe'):
    """
    Esegue una query su PostgreSQL e restituendo DataFrame o JSON (lista di dict). Parametri:
        query (str): Query SQL da eseguire
        format (str): 'dataframe' o 'json'
    """
    try:
        conn = connect_to_postgres()
        if format == 'dataframe':
            df = pd.read_sql_query(query, conn)
            return df
        
        elif format == 'json':
            cur = conn.cursor()
            cur.execute(query)
            columns = [desc[0] for desc in cur.description]
            rows = cur.fetchall()
            return [dict(zip(columns, row)) for row in rows]
        
        else:
            raise ValueError("format deve essere 'dataframe' o 'json'")

    except Exception as e:
        print(f"Errore: {e}")
        return None

    finally:
        if conn:
            conn.close()



In [ ]:

# --- Esempio di utilizzo ---
# Esegui una query SQL
sql_query = "SELECT * FROM users;"

# Ricevere JSON
json_data = query_data(sql_query, format='json')
print('JSON:')
print(json_data)

# Ricevere un DataFrame
df = query_data(sql_query, format='dataframe')
print('Dataframe:')
print(df.head(), '\n') 
# ignore userWARNING


JSON:
[{'id': 1, 'username': 'admin', 'email': 'admin@admin.com', 'password': 'admin', 'created_at': datetime.datetime(2025, 5, 22, 21, 58, 46, 378898)}, {'id': 2, 'username': 'default user', 'email': 'defaut@user.com', 'password': 'user', 'created_at': datetime.datetime(2025, 5, 22, 21, 58, 46, 378898)}]
Dataframe:
   id      username            email password                 created_at
0   1         admin  admin@admin.com    admin 2025-05-22 21:58:46.378898
1   2  default user  defaut@user.com     user 2025-05-22 21:58:46.378898 



C:\Users\User\AppData\Local\Temp\ipykernel_13680\3578338141.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
